In [1]:
import requests
import time
import csv
import pandas as pd
import os
from web3 import Web3
from dotenv import load_dotenv

In [2]:
csv_files = [
    'data/aave/AAVE.csv',
    'data/aave/aEthAAVE.csv',
    'data/aave/aEthUSDC.csv',
    'data/aave/aEthUSDT.csv',
    'data/aave/aEthWETH.csv',
    'data/aave/stkAAVE.csv',
    'data/compound/cCOMP.csv',
    'data/compound/COMP.csv',
    'data/compound/cUSDCv3.csv',
    'data/compound/cUSDTv3.csv',
    'data/compound/cWETHv3.csv',
]

df_list = []

for file in csv_files:
    df = pd.read_csv(file)
    df_list.append(df)
    
combined_df = pd.concat(df_list, ignore_index=True)

addresses = pd.concat([combined_df['from'], combined_df['to']])

unique_addresses = addresses.drop_duplicates()

unique_addresses_df = pd.DataFrame(unique_addresses, columns=['address']).reset_index(drop=True)

In [3]:
csv_node_files = [
    'data/stkAAVE_node_table_jan_2024.csv',
    'data/node_table_jan_2024.csv'
]

df_list = []

for file in csv_node_files:
    df = pd.read_csv(file)
    df_list.append(df)
    
combined_node_df = pd.concat(df_list, ignore_index=True).drop_duplicates(ignore_index=True)

In [4]:
unique_addresses_df = pd.merge(unique_addresses_df, combined_node_df[['address', 'type']], on='address', how='left')

nan_addresses = unique_addresses_df[unique_addresses_df['type'].isna()]

In [10]:
not_na_addresses = unique_addresses_df[unique_addresses_df['type'].notna()]

In [5]:
nan_addresses_first = nan_addresses.iloc[300000:]

In [ ]:
load_dotenv()
api_key = os.getenv('INFURA_API_KEY')

In [6]:
infura_url = f'https://mainnet.infura.io/v3/{api_key}'
web3 = Web3(Web3.HTTPProvider(infura_url))

def is_contract(address):
    code = web3.eth.get_code(Web3.to_checksum_address(address))
    return len(code) > 2

In [7]:
nan_addresses_first['type'] = nan_addresses_first['address'].apply(is_contract)
nan_addresses_first.to_csv('data/node_table_2.csv', index=False)

C:\Users\Sam\AppData\Local\Temp\ipykernel_26160\2446963879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_addresses_first['type'] = nan_addresses_first['address'].apply(is_contract)
